# Jetbot Control with Hand Gestures

## Introduction

Author: Harsh Shroff  
Date: 12/23/2023

This script demonstrates the control of a Jetbot using hand gestures recieved from the client while simultaneously displaying its camera feed. The Jetbot, a robot based on NVIDIA Jetson Nano, is a versatile platform for AI and robotics projects. In this scenario, the script establishes a connection to the Jetbot's camera, processes inputs, and moves the Jetbot accordingly.

## Dependencies

- socket
- threading
- time
- jetbot
- traitlets
- ipywidgets
- opencv-python

## Script Overview

The script begins by importing necessary libraries, including those for handling sockets, threads, time, and Jetbot functionalities. It establishes a connection between the Jetson camera and an IPython display widget for rendering the camera feed.

### Jetbot Initialization

The Jetbot is initialized using the `Robot` class from the `jetbot` library, and a socket is created to listen for gamepad inputs.

### Movement Functions

Several functions are defined to control the Jetbot's movement in different directions:
- `move_forward`
- `move_backward`
- `move_left`
- `move_right`
- `move_360_right`
- `move_360_left`
- `stop`

### ControlThread

A threading class, `ControlThread`, is implemented to control the Jetbot's movement based on gamepad inputs. It listens for specific gamepad inputs and calls the corresponding movement function.

### InputThread

Another threading class, `InputThread`, is responsible for accepting gamepad inputs from a specified port and passing them to the `ControlThread` for handling.

### Execution

The script starts the Jetbot control thread, input thread, and resumes their execution. The Jetbot can be controlled by sending specific commands ("w", "a", "s", "d", "c", "x") to the designated port.

In [1]:
import socket
import threading
import time
from time import sleep
from jetbot import Robot, Camera, bgr8_to_jpeg
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

In [2]:
# Creating a connection between the Jetson camera and a display widget
camera = Camera.instance(width=640, height=480)
image = widgets.Image(format='jpeg', width=640, height=480)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [3]:
robot = Robot()
server_port = 12001
server_port_2 = 1234

# create TCP socket and bind to specified port
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('', server_port))
server_socket.listen()

print("TCP SERVER LISTENING")

TCP SERVER LISTENING


In [4]:
def move_forward():
    robot.forward(0.9)
    time.sleep(0.5)
    robot.forward(0.9)
    time.sleep(0.5)
    robot.stop()

def move_backward():
    robot.backward(0.9)
    time.sleep(0.5)
    robot.backward(0.9)
    time.sleep(0.5)
    robot.stop()

def move_left():
    robot.left(0.5)
    time.sleep(0.3)
    robot.stop()

def move_right():
    robot.right(0.5)
    time.sleep(0.3)
    robot.stop()
    
def move_360_right():
    robot.right(0.9)
    time.sleep(0.2)
    robot.right(0.9)
    time.sleep(0.1)
    robot.stop()
    
def move_360_left():
    robot.left(0.9)
    time.sleep(0.2)
    robot.left(0.9)
    time.sleep(0.1)
    robot.stop()
    
def stop():
    robot.stop()

In [5]:
robot_mode = 'move'

def determine_move(command):
    global robot_mode   
    if robot_mode == 'move':
        move_jetbot(command)

def move_jetbot(button_pressed):
    if not control_bot.new_pressed:
        stop()
    else:
        if button_pressed == "w":
            move_forward()
        elif button_pressed == "s":
            move_backward()
        elif button_pressed == "a":
            move_left()
        elif button_pressed == "d":
            move_right()
        elif button_pressed == "c":
            move_360_right()
        elif button_pressed == "x":
            move_360_left()
    control_bot.new_pressed = 0

In [6]:
class ControlThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(ControlThread, self).__init__(*args, **kwargs)
        self.__flag = threading.Event()
        self.__flag.clear()
        self.button_pressed = ""
        self.new_pressed = 0
        self.mode_pressed = 0

    def pause(self):
        self.__flag.clear()

    def resume(self):
        self.__flag.set()

    def run(self):
        while 1:
            self.__flag.wait()

            # Calls functions based on the variable set in Input thread
            if self.new_pressed:
                determine_move(self.button_pressed)
            elif self.mode_pressed:
                change_mode(self.button_pressed)
            if not self.new_pressed and not self.mode_pressed:
                self.pause()
                
# Instantiate and start the multi-threaded servo control thread.
control_bot = ControlThread()
control_bot.start()

In [7]:
class InputThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(InputThread, self).__init__(*args, **kwargs)
        self.__flag = threading.Event()
        self.__flag.clear()

    def pause(self):
        self.__flag.clear()

    def resume(self):
        self.__flag.set()

    def run(self):
        while 1:
            conn, addr = server_socket.accept()
            with conn:
                print("Connected to: ")
                print(addr)
                while 1:
                    self.__flag.wait()
                    message, client_address = conn.recvfrom(2048)
                    mess = message.decode()

                    # Look for specific input to steer the Jetank accordingly
                    if mess in {"w", "a", "s", "d", "c", "x"}:
                        print(mess)
                        control_bot.button_pressed = mess
                        control_bot.new_pressed = 1
                        control_bot.resume()

# Instantiate the gamepad input thread and start reading information
input_threading = InputThread()
input_threading.start()
input_threading.resume()

Connected to: 
('10.200.135.101', 64052)
w
w
w
s
s
s
s
s
a
a
a
a
a
a
a
d
d
d
d
c
w
d
d
a
a
a
w
w
s
s
c
c
c
c
c
c
c
x
x
x
x
w
w
a
a
a
d
c
c
c
w
w
w
s
x
c
s
w
c
c
c
c
c
c
c
c
c
c
c
